# NL HW 1 (2021)

## Дарья Родионова БКЛ182

### Keywords Extraction

In [159]:
import re

import pandas as pd
import numpy as np

import spacy
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

nltk.download(['punkt', 'stopwords'])

import RAKE
from summa.textrank import textrank
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ddrodionova/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ddrodionova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 1. Подготовка мини-корпуса

В качестве мини-корпуса я выбрала [датасет](https://www.kaggle.com/achintyatripathi/news-dataset-18920) с Kaggle, который в основном содержит индийские новости про технологии и образование. 

In [32]:
data = pd.read_csv('news_dataset.csv')

In [33]:
data.head(10)

,Unnamed: 0,Unnamed: 0.1,link,text,title,date,keywords,summary,title_summary
0,0,0,http://techcrunch.com/2020/09/07/vodafone-idea...,"Vodafone Idea, one of the largest telecom oper...",Indian telecom giant Vodafone Idea rebrands as...,2020-09-07 00:00:00,"['rebrands', 'idea', 'vi', 'giant', 'brand', '...","Vodafone Idea, one of the largest telecom oper...",Indian telecom giant Vodafone Idea rebrands as...
1,1,1,http://techcrunch.com/2020/09/16/facebook-addr...,"At the beginning of the previous decade, Faceb...",Facebook addresses political controversy in In...,2020-09-16 00:00:00,"['opportunities', 'whatsapp', 'controversy', '...",Politicians in the country today heavily rely ...,Facebook addresses political controversy in In...
2,2,2,http://techcrunch.com/2020/09/14/youtube-launc...,"As TikTok’s fate in the U.S. remains murky, Yo...","YouTube launches its TikTok rival, YouTube Sho...",2020-09-14 00:00:00,"['rival', 'video', 'feature', 'access', 'youtu...","As TikTok’s fate in the U.S. remains murky, Yo...","YouTube launches its TikTok rival, YouTube Sho..."
3,3,3,http://techcrunch.com/2020/09/09/groww-an-inve...,Even as more than 150 million people are using...,"Groww, an investment app for millennials in In...",2020-09-09 00:00:00,"['world', 'yc', 'continuity', 'stocks', 'groww...","YC Continuity, the growth-stage investment fun...","Groww, an investment app for millennials in In..."
4,4,4,http://techcrunch.com/2020/09/15/lanzatech-is-...,As part of the continuing global rollout of La...,LanzaTech is developing a small-scale waste bi...,2020-09-15 00:00:00,"['production', 'distributed', 'developing', 's...",As part of the continuing global rollout of La...,LanzaTech is developing a small-scale waste bi...
5,5,5,http://techcrunch.com/2020/09/15/facebook-anno...,More than a third of small and medium-sized bu...,Facebook announces $4.3 million grant for smal...,2020-09-15 00:00:00,"['businesses', 'pandemic', 'support', 'cash', ...",The social giant today announced a grant of $4...,Facebook announces $4.3 million grant for smal...
6,6,6,http://techcrunch.com/2020/09/14/indian-e-comm...,"CashKaro, one of the leading cashback and coup...",Indian e-commerce deals site CashKaro gets $10...,2020-09-14 00:00:00,"['site', 'korea', 'shoppers', 'gets', 'series'...",TechCrunch last covered CashKaro five years ag...,Indian e-commerce deals site CashKaro gets $10...
7,7,7,http://techcrunch.com/2020/09/15/virtual-event...,"Airmeet, a startup that offers a platform to h...",Virtual events platform Airmeet raises $12M – ...,2020-09-15 00:00:00,"['platform', 'global', 'virtual', 'capital', '...",Sequoia Capital India led the $12 million Seri...,Virtual events platform Airmeet raises $12M – ...
8,8,8,http://techcrunch.com/2020/09/15/indian-decaco...,Since India enforced a lockdown across the cou...,Indian decacorn Byju’s CEO talks about future ...,2020-09-15 00:00:00,"['pandemic', 'talks', 'expansion', 'coronaviru...",It took the startup about four and a half year...,Indian decacorn Byju’s CEO talks about future ...
9,9,9,http://techcrunch.com/2020/09/07/silver-lake-l...,Byju’s has raised $500 million in a new financ...,Silver Lake leads $500 million investment roun...,2020-09-07 00:00:00,"['learning', 'students', 'statement', 'lake', ...",Byju’s has raised $500 million in a new financ...,Silver Lake leads $500 million investment roun...


Возьмём первые 10 статей и посмотрим, хватает ли нам токенов. 

In [34]:
news_dataset = data[:10]

In [35]:
news_dataset.text.apply(lambda x: len(x.split())).sum()

6498

Выбросим ненужные колонки.

In [36]:
news_dataset.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'link', 'title', 'date', 'summary', 'title_summary'], inplace=True)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [37]:
news_dataset

,text,keywords
0,"Vodafone Idea, one of the largest telecom oper...","['rebrands', 'idea', 'vi', 'giant', 'brand', '..."
1,"At the beginning of the previous decade, Faceb...","['opportunities', 'whatsapp', 'controversy', '..."
2,"As TikTok’s fate in the U.S. remains murky, Yo...","['rival', 'video', 'feature', 'access', 'youtu..."
3,Even as more than 150 million people are using...,"['world', 'yc', 'continuity', 'stocks', 'groww..."
4,As part of the continuing global rollout of La...,"['production', 'distributed', 'developing', 's..."
5,More than a third of small and medium-sized bu...,"['businesses', 'pandemic', 'support', 'cash', ..."
6,"CashKaro, one of the leading cashback and coup...","['site', 'korea', 'shoppers', 'gets', 'series'..."
7,"Airmeet, a startup that offers a platform to h...","['platform', 'global', 'virtual', 'capital', '..."
8,Since India enforced a lockdown across the cou...,"['pandemic', 'talks', 'expansion', 'coronaviru..."
9,Byju’s has raised $500 million in a new financ...,"['learning', 'students', 'statement', 'lake', ..."


### 2. Ручная разметка ключевых слов и создание эталона

При выделение ключевых слов я в основном ориентировалась на слова, данные автором датасета. Мне кажется, что некоторые из них сложно назвать ключевыми словами: очень много односложных слов, которые ничего не говорят о тексте статьи (например, абстрактные wolrd или глаголы в странной форме типа gets). Вообще есть такое ощущение, что эти ключевые слова когда-то были размечены копьютером. 

In [44]:
my_keywords = []
for ind, text in news_dataset.iterrows():
    print('****** TEXT ******')
    print('\n', text['text'])
    print('\n****** KEYWORDS ******')
    print('\n', text['keywords'])
    keywords = input('Keywords: ').split(', ')
    my_keywords.append(keywords)

****** TEXT ******

 Vodafone Idea, one of the largest telecom operators in India, has rebranded as ‘Vi’ as it looks for a “fresh start” three years after the British telecom giant Vodafone Group’s India business and billionaire Kumar Mangalam Birla’s Idea Cellular merged in the country.

“As the integration of two businesses is now complete, it’s time for a fresh start. That’s why we believe that now is the perfect time to launch Vi, one company which provides the strength of Vodafone India and Idea,” Vodafone Group CEO Nick Read said at a virtual conference on Monday.

Vodafone Idea, once the largest telecom operator in the country with over 400 million subscribers, has lost more than 100 million subscribers in recent years to new comer Jio Platforms in recent years.

India’s richest man Mukesh Ambani’s telecom venture has become the largest telecom operator with nearly 400 million subscribers with its cut-rate mobile data tariffs. Jio Platforms has also attracted over $20 billion in

Keywords: controversy, facebook, partnership, politics
****** TEXT ******

 As TikTok’s fate in the U.S. remains murky, YouTube today announced the launch of a new short-form video experience it’s calling YouTube Shorts. The feature will allow users, initially in India, to upload 15-second or less short-form videos using a new set of creator tools, including a multi-segment camera, similar to TikTok, speed controls and a timer and a countdown feature. The videos can also be set to music, thanks to YouTube’s access to a large library of songs that it says will continue to grow over time.

The multi-segment camera lets users string together multiple video clips into one short clip. The other controls enable YouTube creators to be more creative than if they simply pressed a “record” button.

These are tools common to the TikTok video recording experience today and are the same features that Instagram recently cloned for its own TikTok rival, known as Reels.

The feature will be prominentl

Keywords: carbon dioxide emissions, gasifier, decarbonization
****** TEXT ******

 More than a third of small and medium-sized businesses on Facebook in India expect cash flow to be a challenge for them as they navigate the coronavirus pandemic in the next few months, according to a report by Organisation for Economic Co-operation and Development (OECD) and the World Bank.

Facebook, which reaches nearly every internet user in India and which collaborated with OECD and World Bank on the report, wants to help. The social giant today announced a grant of $4.3 million for more than 3,000 small businesses across Delhi, Gurgaon, Mumbai, Hyderabad and Bangalore (Indian cities where the company has its offices).

In an interview with TechCrunch, Ajit Mohan, head of Facebook India, said the grant includes both cash and ad credits, with cash constituting the larger share. These businesses don’t have to advertise on Facebook to be eligible for the grant, he said. Businesses can apply for the gra

Keywords: cashback, e-commerce, investment, korea
****** TEXT ******

 Airmeet, a startup that offers a platform to host virtual events, said on Tuesday it has raised $12 million in a new financing round as the Bangalore-headquartered firm demonstrates accelerating growth in its user base.

Sequoia Capital India led the $12 million Series A financing round in one-year-old Airmeet. Redpoint Ventures and existing investors Accel India, Venture Highway, Global Founders Capital (GFC) and Gokul Rajaram (Caviar Lead at DoorDash) also participated in the round.

The new round values Airmeet at about $50 million, more than double of what it was valued in March, when it raised $3 million, according to a person familiar with the matter.

Airmeet allows users and businesses to host interactive virtual events. Its platform intuitively replicates aspects of a physical event, offering a backstage, grouping people to a table, allowing participants to network with each other and even enabling event or

Keywords: pandemic, lockdown, students, international markets, business models
****** TEXT ******

 Byju’s has raised $500 million in a new financing round that values the Indian online learning platform at $10.8 billion, a source familiar with the matter told TechCrunch.

The round was led by Menlo Park-headquartered PE firm Silver Lake, and existing investors Tiger Global, General Atlantic and Owl Ventures have also participated in it, India’s second most valuable startup Byju’s said in a statement, but declined to reveal the size of the round and its valuation.

“We are excited to welcome a strong partner like Silver Lake to the BYJU’S family,” said Byju Raveendran, co-founder and CEO of the eponymous startup in a statement. Raveendran will be appearing at Disrupt 2020 next week.

“We are fortunate to be in a sector of positive relevance during this crisis. This has brought online learning to the forefront and is helping parents, teachers and students experience and understand its v

In [47]:
news_dataset['my_keywords'] = my_keywords

<ipython-input-47-f83edf3f42ef>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_dataset['my_keywords'] = my_keywords


In [59]:
given_keywords = news_dataset.keywords.to_list()
new_keywords = news_dataset.my_keywords.to_list()

В качестве эталона я решила объединить данные изначально ключевые слова с размеченными мною.

In [62]:
gold_keywords = []
for g, m in zip(given_keywords, new_keywords):
    union = list(set(g) | set(m))
    gold_keywords.append(union)

In [63]:
news_dataset['gold_keywords'] = gold_keywords

<ipython-input-63-b0cc52109e27>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_dataset['gold_keywords'] = gold_keywords


### Автоматические извлечение ключевых слов

In [86]:
nlp = spacy.load('en_core_web_sm')
stop_words = stopwords.words('english')

In [75]:
def preprocess(text, nlp):
    tokens = nlp(text)
    words = ' '.join([w.lemma_ for w in tokens if w.lemma_.isalpha()])
    return words

Лемматизирую сначала только тексты, так потом удобнее будет работать.

In [79]:
news_dataset['texts_lemmatized'] = news_dataset.text.apply(lambda x: preprocess(x, nlp))

<ipython-input-79-58cd47963b9a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_dataset['texts_lemmatized'] = news_dataset.text.apply(lambda x: preprocess(x, nlp))


#### Rake

In [87]:
rake = RAKE.Rake(stop_words)

In [175]:
def keywords_rake(text, rake):
    rake_output = rake.run(text, 
                           maxWords=3, minFrequency=2)
    rake_output = [k[0] for k in rake_output]
    return ', '.join(rake_output)

Как мне показалось, Rake лучше справляется на обычных текстах. 

In [176]:
news_dataset['rake_raw'] = news_dataset.apply(lambda x: keywords_rake(x['text'], rake), axis=1)
news_dataset['rake_lem'] = news_dataset.apply(lambda x: keywords_rake(x['texts_lemmatized'], rake), axis=1)                                              

<ipython-input-176-a299243e45f6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_dataset['rake_raw'] = news_dataset.apply(lambda x: keywords_rake(x['text'], rake), axis=1)
<ipython-input-176-a299243e45f6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_dataset['rake_lem'] = news_dataset.apply(lambda x: keywords_rake(x['texts_lemmatized'], rake), axis=1)


In [96]:
news_dataset['rake_raw']

0    previous brand names, largest telecom operator...
1    content policy, make sure, add value, mohan sa...
2    new watch experience, shorts camera, youtube s...
3    mutual funds, investment world, allow users, r...
4    scale, distributed, lanzatech, holmgren, india...
5    gift cards, world bank, small businesses, ” sa...
6    e-commerce sites, korea investment partners, s...
7    said mangal, airmeet, said, use, platform, sta...
8    school-going students, international markets, ...
9    round, techcrunch, year, valuation, india, byj...
Name: rake_raw, dtype: object

In [95]:
news_dataset['rake_lem']

0    large telecom operator, million subscriber, in...
1    million internet user, content policy, add val...
2    new watch experience, shorts camera, time yout...
3    allow user, buy stock, groww, india, startup, ...
4      india, result, fuel, chemical, roll, production
5    gift card, small business, facebook, india, bu...
6    series b, partner site, cashkaro, month, compa...
7    airmeet, use, platform, startup, work, usage, ...
8    school go student, primary capital, byju, star...
9    raise capital, round, lead, byju, value, india...
Name: rake_lem, dtype: object

#### TextRank

TextRank просто ужасно справляется на нелемматизированных текстах (выделяет абсолютно все формы одного слова), поэтому после проверки я удалила результаты его работы из датасета и оставила только один вариант.

In [178]:
news_dataset['textrank'] = news_dataset.texts_lemmatized.apply(lambda x:
                                                  set(textrank(x, summarize_by=None).split('\n')))

<ipython-input-178-e54ae0a73e63>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_dataset['textrank'] = news_dataset.texts_lemmatized.apply(lambda x:


#### Tf–Idf

In [209]:
vectorizer = TfidfVectorizer(stop_words=stop_words, ngram_range=(1, 3))
tfidf = vectorizer.fit_transform(news_dataset.texts_lemmatized)
features = np.array(vectorizer.get_feature_names())
sorted_words = np.argsort(tfidf.toarray())

In [211]:
tfidf_keywords = []

for i in sorted_words:
    idx = i[::-1]
    tfidf_keywords.append(features[idx][:10].tolist())

Аналогично прогоняю Tf–Idf на лемматизированных текстах.

In [212]:
news_dataset['tfidf_lem'] = tfidf_keywords

<ipython-input-212-66a0be778450>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_dataset['tfidf_lem'] = tfidf_keywords


In [121]:
news_dataset.tfidf_lem

0    [vodafone, idea, vodafone idea, telecom, birla...
1    [facebook, india, say, company, whatsapp, moha...
2    [youtube, video, tiktok, shorts, feature, shor...
3    [groww, fund, stock, mutual, keshre, financial...
4    [lanzatech, holmgren, produce, gasifier, scale...
5    [business, facebook, grant, small business, sm...
6    [cashkaro, commerce, site, earnkaro, cashback,...
7    [airmeet, event, mangal, say, industry, host, ...
8    [byju, startup, raveendran, student, market, s...
9    [byju, silver, silver lake, lake, student, val...
Name: tfidf_lem, dtype: object

#### 4. Составление шаблонов

После просмотра эталонных ключевых слов, я решила задать шаблоны вручную, потому что они достаточно простые и понятные.

In [224]:
def check_pattern(text):
    patterns = ['NOUN', 'NOUN NOUN', 'NOUN NOUN NOUN', 'ADJ NOUN NOUN', 
                'ADJ NOUN', 'ADJ', 'VERB', 'ADV VERB', 'ADV', 'ADJ VERB NOUN']
    doc = nlp(text)
    pos = ' '.join([token.pos_ for token in doc])
    for word in doc:
        if word.pos_ in patterns:
            return text

In [190]:
def filter_keywords(keywords):
    filtered = [check_pattern(w) for w in keywords]
    return set([w for w in filtered if w])

#### 5. Оценка метрик

In [184]:
def evaluate(y_pred, y_true):
    y_pred = y_pred.apply(lambda x: set([''.join(w.split()) for w in x])).values
    y_true = y_true.apply(lambda x: set([''.join(w.split()) for w in x])).values

    tp = np.apply_along_axis(func1d=lambda x: len(x[0]),
                             axis=1,
                             arr=np.expand_dims(y_pred & y_true, 1))

    fp = np.apply_along_axis(func1d=lambda x: len(x[0]),
                             axis=1,
                             arr=np.expand_dims(y_pred - y_true, 1))
    
    fn = np.apply_along_axis(func1d=lambda x: len(x[0]),
                             axis=1,
                             arr=np.expand_dims(y_true - y_pred, 1))

    macro_precision = (tp / (tp + fp + 1e-25)).mean()
    macro_recall = (tp / (tp + fn)).mean()
    macro_f1 =  (2 / (macro_recall ** -1 + macro_precision ** -1)).mean()

    tp = tp.sum()
    fp = fp.sum()
    fn = fn.sum()

    micro_precision = tp / (tp + fp) 
    micro_recall = tp / (tp + fn)
    micro_f1 =  2 / (micro_recall ** -1 + micro_precision ** -1)

    results = {'precision': {'macro': macro_precision,
                             'micro': micro_precision},
               'recall': {'macro': macro_recall,
                          'micro': micro_recall},
               'f1': {'macro': macro_f1,
                      'micro': micro_f1}
               }

    return pd.DataFrame(results)

#### Rake

<p>С Rake результаты очень плачевные :(</p>
Это связано с тем, что он совсем иначе определил ключевые слова, и ни одно из них не совпало с эталоном :(

In [225]:
evaluate(news_dataset.rake_raw, news_dataset.gold_keywords)

<ipython-input-184-ea128bf6a19e>:19: RuntimeWarning: divide by zero encountered in double_scalars
  macro_f1 =  (2 / (macro_recall ** -1 + macro_precision ** -1)).mean()
<ipython-input-184-ea128bf6a19e>:27: RuntimeWarning: divide by zero encountered in double_scalars
  micro_f1 =  2 / (micro_recall ** -1 + micro_precision ** -1)


,precision,recall,f1
macro,0.0,0.0,0.0
micro,0.0,0.0,0.0


In [226]:
evaluate(news_dataset.rake_raw.apply(filter_keywords), news_dataset.gold_keywords)

<ipython-input-184-ea128bf6a19e>:19: RuntimeWarning: divide by zero encountered in double_scalars
  macro_f1 =  (2 / (macro_recall ** -1 + macro_precision ** -1)).mean()
<ipython-input-184-ea128bf6a19e>:27: RuntimeWarning: divide by zero encountered in double_scalars
  micro_f1 =  2 / (micro_recall ** -1 + micro_precision ** -1)


,precision,recall,f1
macro,0.0,0.0,0.0
micro,0.0,0.0,0.0


Для наглядности посмотрим на первый текст.

In [242]:
news_dataset.gold_keywords[0]

['vodafone',
 'largest',
 'brand',
 'birla',
 'telecom',
 'techcrunch',
 'idea',
 'vi',
 'wireless technology',
 'billion',
 'rebrands',
 'company',
 'india',
 'giant',
 'indian']

In [243]:
news_dataset.rake_raw[0]

'previous brand names, largest telecom operator, vodafone idea, recent years, india, time, government, country, &'

#### TextRank

In [227]:
evaluate(news_dataset.textrank, news_dataset.gold_keywords)

,precision,recall,f1
macro,0.216179,0.337948,0.263684
micro,0.202658,0.331522,0.251546


In [228]:
evaluate(news_dataset.textrank.apply(filter_keywords), news_dataset.gold_keywords)

,precision,recall,f1
macro,0.146172,0.148841,0.147494
micro,0.129808,0.146739,0.137755


#### Tf-Idf

Tf-Idf справился лучше всех.

In [229]:
evaluate(news_dataset.tfidf_lem, news_dataset.gold_keywords)

,precision,recall,f1
macro,0.55,0.303206,0.390909
micro,0.55,0.298913,0.387324


In [230]:
evaluate(news_dataset.tfidf_lem.apply(filter_keywords), news_dataset.gold_keywords)

,precision,recall,f1
macro,0.429167,0.142272,0.213701
micro,0.448276,0.141304,0.214876


#### Final remarks

В целом всё очень плохо, а с наложением шаблонов ситуация не улучшается.

Главная проблема, на мой взгляд, заключается в размеченных авторов датасета словах. Как я уже и сказала, есть ощущение, что их тоже разметила какая-то программа. Имеет смысл составить новый эталон либо из пересечения этих слов и своих, либо избавиться от них совсем, потому что многие из них не сообщают ключевую информацию о текстах совсем.

Не уверена, что это решило бы проблему с Rake, потому что, по-моему, он тоже выделил довольно странные сочетания, которые далеко не являются ключевыми. Вообще тексты довольно маленькие, в них в целом не так уж и много слов, чтобы с уверенностью можно было сказать, что какие-то из них более ключевые, чем другие, основываясь только на частотности. 

В целом подобная оценка ключевых слов довольно субъективна как и со стороны людей, так и со стороны алгоритмов. Мне кажется, что полагаться на частотность не всегда верно, иногда ключевые слова являются результатом обобщения некоторых часто встречаемых слов. Но такая работа обычно проводится людьми (или WordNet), а данные методы, к сожалению, опираются на только на частотность.